In [1]:
import os
import cv2
import wget
from tqdm.notebook import tqdm
import pandas as pd
import random as rd
import numpy as np

In [6]:
df = pd.read_csv('trailers.csv')

In [7]:
df.head()

,trailer_title,url,imdb_id,year,rating,votes,genre
0,THE BIG PARADE trailer,https://archive.org/download/TheBigParadeTrail...,15624,1925,7.9,6148.0,Drama|Romance|War
1,ALL QUIET ON THE WESTERN FRONT 1930 trailer,https://archive.org/download/AllQuietOnTheWest...,20629,1930,8.0,56948.0,Drama|War
2,THE OREGON TRAIL trailer,https://archive.org/download/TheOregonTrailTra...,20691,1926,7.2,3353.0,Adventure|Romance|Western
3,FIVE STAR FINAL trailer,https://archive.org/download/FiveStarFinalTrai...,21873,1931,7.2,1790.0,Crime|Drama
4,LITTLE CAESAR trailer,https://archive.org/download/LittleCaesarTrail...,21079,1931,7.3,11958.0,Action|Crime|Drama|Romance


Now i am going to extract the subsets from 6.1 the same way, i will use a little bigger sample now with a number o n = 10

In [8]:
n = 10
trailers1920_40 = trailers[(trailers.year > 1920) & (trailers.year < 1940)].sample(n)
trailers1960_80 = trailers[(trailers.year > 1960) & (trailers.year < 1980)].sample(n)
trailers2000 = trailers[trailers.year > 2000].sample(n)

In [9]:
def dl_sample(df, folder):
    if not os.path.exists(folder):
        os.mkdir(folder)
    
    video_paths = []
    for video in df.itertuples():
        video_url = video.url
        output_path = folder + video.trailer_title + '.mp4'
        filename = wget.download(video_url, out=output_path)
        video_paths.append(output_path)
        
    return video_paths

In [10]:
trailer1920 = dl_sample(trailers1920_40, 'vid_1920/')
trailer1960 = dl_sample(trailers1960_80, 'vid_1960/')
trailer2000 = dl_sample(trailers2000, 'vid_2000/')

In [12]:
trailer_samples = pd.concat([trailers1920_40,trailers1960_80,trailers2000])# all the trailers in one list
print(len(trailer_samples))
trailer_samples.head()

30


,trailer_title,url,imdb_id,year,rating,votes,genre
76,IDIOT'S DELIGHT trailer,https://archive.org/download/IdiotsDelightTrai...,31473,1939,6.5,1289.0,Comedy|Drama|Musical|Romance
61,THE GOOD EARTH trailer,https://archive.org/download/TheGoodEarthTrail...,28944,1937,7.5,4474.0,Drama|Romance
86,JUAREZ trailer,https://archive.org/download/JuarezTrailer/Jua...,31516,1939,7.1,1963.0,Biography|Drama|History|Romance
58,THE LIFE OF EMILE ZOLA Trailer,https://archive.org/download/THELIFEOFEMILEZOL...,29146,1937,7.2,7066.0,Biography|Drama
46,THEODORA GOES WILD trailer,https://archive.org/download/TheodoraGoesWildT...,28355,1936,7.2,2123.0,Comedy|Romance


In [77]:
#I find all the scenes from the movies using the function from the manual
from scenedetect import VideoManager
from scenedetect import SceneManager

from scenedetect.detectors import ContentDetector
from scenedetect.scene_manager import generate_images

def find_scenes(video_path, threshold=30.0):
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(
        ContentDetector(threshold=threshold))
    base_timecode = video_manager.get_base_timecode()
    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager, show_progress=False)    
    return scene_manager.get_scene_list(base_timecode)

In [78]:
#I am using the face classification to detect the faces from manual
face_classification = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') # load the classifier only once!

In [79]:
#I am loading the emotions classifier also from the manual
emotion_classifier = load_model('fer2013_mini_XCEPTION.102-0.66.hdf5') # load this only once! (not in a loop)
EMOTION_OFFSETS = (0, 0)
INPUT_SHAPE_EMOTION = emotion_classifier.input_shape[1:3]
labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

NameError: name 'load_model' is not defined

Now i have to create the loop to find first the faces and then the emotions from the 3 subsets that i have i am going to create 6 variances 2 for every subset(number of faces and emotions)

In [66]:
#firstly i am going to loop through the 1920 until 1940 videos 
tr_faces_1920 = pd.DataFrame()
tr_emotions_1920 = pd.DataFrame()

In [75]:
#i have to loop through all the videos
for i in range(len(trailers1920_40)): #to get the trailers
    tr = trailers1920_40.iloc[i]
    tr_url = tr["url"]
    tr_file = tr_url.split("/")[-1]
    if os.path.isfile(str(tr_file)):
        print (str(tr_file))
    else:
        print(f"error with {tr_file}")
     # get frames
    filename = tr_file
    if filename not in tr_faces_1920.index:
        cap = cv2.VideoCapture(filename)
        frames = []
        scene_list = find_scenes(filename, threshold=30)
        for start_time, end_time in scene_list:
            duration = end_time - start_time
            frame = (start_time.get_frames() + int(duration.get_frames() / 2))
            cap.set(cv2.CAP_PROP_POS_FRAMES,frame)
            ret, frame = cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(frame)

        # get amount of faces in frame
        frame_faces = []
        emotions_probas = []
        for frame in frames:
            gray_image = frame
            faces = face_classification.detectMultiScale(gray_image, 1.3, 5) # detect the faces 
            n_faces = len(faces) # get the number of faces
            frame_faces.append(n_faces)
            # for each frame with faces, I want to get the emotions
            if n_faces > 0:
                for face_coordinates in faces:
                    x1, x2, y1, y2 = apply_offsets(face_coordinates, EMOTION_OFFSETS) 
                    face_img = gray_image[y1:y2, x1:x2] # only get the face
                    face_img = cv2.resize(face_img, (INPUT_SHAPE_EMOTION))
                    face_img = face_img.astype('float32') / 255.0 # pre-processing 
                    face_img = face_img - 0.5 # pre-processing specific to the emotion classifier
                    face_img = face_img * 2.0 # pre-processing specific to the emotion classifier
                    face_img = np.expand_dims(face_img, 0) # batch of one
                    face_img = np.expand_dims(face_img, -1) # pre-processing specific to the emotion classifier
                    probas = emotion_classifier.predict(face_img).squeeze()
                    # add emotions of all faces to a list
                    emotions_probas.append(probas)
        face_described = pd.DataFrame(frame_faces).describe()
        face_described_t = face_described.transpose()
        face_described_t.columns = [f"faces_in_frame_{desc}" for desc in face_described_t.columns]
        face_described_t.index = [str(tr_file)]
        trailer_faces = pd.concat([trailer_faces, face_described.transpose()])
        
        # make a df from probabilities
        emotions_probas_df = pd.DataFrame(emotions_probas, columns = labels)
        described_emotions = pd.DataFrame()
        for label in labels:
            described_emotion = pd.DataFrame(emotions_probas_df[label].describe())
            described_emotion_t = described_emotion.transpose()
            described_emotion_t.columns = [f"{label}_{desc}" for desc in described_emotion_t.columns]
            described_emotions = pd.concat([described_emotions, described_emotion_t.reset_index(drop=True)], axis=1)
        described_emotions.index = [filename]
        tr_emotions_1920 = pd.concat([tr_emotions_1920, described_emotions])        
        
        

error with IdiotsDelightTrailer.mp4


OSError: Video file(s) not found.

In [76]:
 for i in range(len(trailers1920_40)):
    tr = trailers1920_40.iloc[i]
    print(tr["trailer_title"])
    
    tr_url = tr["url"]
    tr_file = tr_url.split("/")[-1]
    if not os.path.isfile(tr_file):
        os.system("wget " + tr_url)

IDIOT'S DELIGHT trailer
THE GOOD EARTH trailer
JUAREZ trailer
THE LIFE OF EMILE ZOLA Trailer
THEODORA GOES WILD trailer
THE MUMMY'S HAND trailer
BLACK LEGION trailer
HEROES FOR SALE trailer.
GOLD DIGGERS OF 1935 trailer
NINOTCHKA trailer.
